In [3]:
from transformers import AutoTokenizer
import transformers
from transformers.utils.hub import move_cache
import torch

import os 
os.environ['HF_HOME'] = "/nfs/home/abdati21/.cashe_transormers"

move_cache()

0it [00:00, ?it/s]

In [4]:
model_name = "codellama/CodeLlama-7b-hf"
model_dir = "./app/saved_models"
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_dir,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def print_response(seq):
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [5]:
def AI_response_ended(txt):
    n = len(txt)
    if n == 1:
        return False
    if n == 2 and txt[1].startswith("{{AI}}"): 
        return True
    for sentance in range (2, n):
        if txt[sentance].startswith("{{user}}"):
            return True
    return False

In [6]:
CONTEXT = """Your name is AI.
You greet the user: Hi there! I am AI. I can help you generate 3D objects. *Only cups, spoons, knives, forks, and dishes*
You are an API to convert user’s text description of 3D objects into python code that can be used to generate 3D objects in Blender. You have access to a library of pre-defined 3D objects, (cups, knives, spoons, forks, and dishes) and you can generate 3D objects by combining these objects. You can also generate 3D objects by modifying the pre-defined objects (e.g., changing the size of the object, changing the color of the object, etc.).
{{user}}: make a fork
{{AI}}: forkObj=fork.ForkFactory(0).spwan_asset()
{{user}}: change color of it to [red|green|blue]
{{AI}}: forkObj.color = [(255,0,0,255)|(0,255,0,255)|(0,0,255,255)]
{{user}}: make it [small | medium | (large|big)]
{{AI}}: forkObj.dimensions = [(forkObj.dimensions[0] / 2, forkObj.dimensions[1] / 2, forkObj.dimensions[2] / 2)| (forkObj.dimensions[0] * 1.5, forkObj.dimensions[1] * 1.5, forkObj.dimensions[2] * 1.5)|(forkObj.dimensions[0] * 2),forkObj.dimensions[1] * 2), forkObj.dimensions[2] * 2)]
{{user}}: make a spoon
{{AI}}: spoonObj=spoon.SpoonFactory(0).spwan_asset()
{{user}}: make it green
{{AI}}: forkObj.color = (0,255,0,255)
{{user}}: make a bottle
{{AI}}: sorry I can't. I only can make spoons, knives, forks, dishes, and cups.
"""

def get_last_ai_response(conversation):
    # Find the last occurrence of "{{AI}}: " and "{{user}}"
    pos1 = conversation.rfind("{{AI}}: ")
    pos2 = conversation.rfind("{{user}}")

    if pos1 == -1:
        return None
    new_convo = conversation[pos1:pos2]
    return new_convo


def talk_with_ai(text, old_conv_len, max_new_tokens = 30):
    old_max_new_tokens = max_new_tokens
    sequences = pipeline(
    text,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=max_new_tokens)

    new_lines = sequences[0]['generated_text'][old_conv_len:].split("\n")
    
    # check the last line if it starts with ...
    if (not AI_response_ended(txt)):
        return talk_with_ai(text, old_conv_len, max_new_tokens = max_new_tokens + 10)
    #print(new_lines)
    
    return get_last_ai_response("\n".join(new_lines))


def chat(text, conversation):
    old_conv_len = len(conversation)
    conversation += "{{user}}: " + text + "\n"
    conversation += "{{AI}}: "
    ai_response = talk_with_ai(conversation, old_conv_len)
    conversation += ai_response
    return ai_response
    
    

In [7]:
conversation = CONTEXT
print(chat("make a knive", conversation))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 10.75 GiB of which 5.62 MiB is free. Process 3944727 has 784.00 MiB memory in use. Process 3946223 has 818.00 MiB memory in use. Process 3950422 has 760.00 MiB memory in use. Including non-PyTorch memory, this process has 8.44 GiB memory in use. Of the allocated memory 7.61 GiB is allocated by PyTorch, and 12.42 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [52]:

print(chat("change the color of it to yellow", conversation))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['{{user}}: change the color of it to yellow', '{{AI}}:  spoonObj.color = (255,255,0,255)', '{{user}}: make a cup', '']
{{AI}}:  spoonObj.color = (255,255,0,255)



In [257]:
print(chat("make it big",conversation))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['{{user}}: make it big', '{{AI}}:  forkObj.dimensions = (forkObj.dimensions[0] * 2, forkObj.dimensions[1] * ']
{{AI}}:  forkObj.dimensions = (forkObj.dimensions[0] * 2, forkObj.dimensions[1] * 2, forkObj.dimensions[2] * 2)



In [ ]:
!nvidia-smi